In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

import random
from tqdm import tqdm
import esm

In [2]:
seed_value = 42
random.seed(seed_value)                         # 设置 random 模块的随机种子
np.random.seed(seed_value)                      # 设置 numpy 模块的随机种子
torch.manual_seed(seed_value)                   # 设置 PyTorch 中 CPU 的随机种子
#tf.random.set_seed(seed_value)                 # 设置 Tensorflow 中随机种子
if torch.cuda.is_available():                   # 如果可以使用 CUDA，设置随机种子
    torch.cuda.manual_seed(seed_value)          # 设置 PyTorch 中 GPU 的随机种子
# 检测GPU是否可用
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 加载ESM模型

In [3]:
# Load ESM-2 model
esm_model, alphabet = esm.pretrained.esm2_t6_8M_UR50D() #320d
batch_converter = alphabet.get_batch_converter()
esm_model.eval()  # disables dropout for deterministic results

ESM2(
  (embed_tokens): Embedding(33, 320, padding_idx=1)
  (layers): ModuleList(
    (0-5): 6 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=320, out_features=320, bias=True)
        (v_proj): Linear(in_features=320, out_features=320, bias=True)
        (q_proj): Linear(in_features=320, out_features=320, bias=True)
        (out_proj): Linear(in_features=320, out_features=320, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=320, out_features=1280, bias=True)
      (fc2): Linear(in_features=1280, out_features=320, bias=True)
      (final_layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=120, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (emb_layer_norm_after): LayerNorm((320,), eps=1e-05, elementwis

In [4]:
def ESM_feature(sequence):
    data = []
    for i in tqdm(sequence):
        row = (i,i)
        data.append(row)

    batch_labels, batch_strs, batch_tokens = batch_converter(data)
    batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

    with torch.no_grad():
        results = esm_model(batch_tokens, repr_layers=[6], return_contacts=True)
    token_representations = results["representations"][6]
# Generate per-sequence representations via averaging
# NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
    sequence_representations = []
    for i, tokens_len in enumerate(batch_lens):
        sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
    return sequence_representations

In [5]:
df=pd.read_csv("test192.txt", na_filter=False)

In [8]:
df.head(5)

,sequence
0,VSV
1,INEGSLLPH
2,LRP
3,IVY
4,IWW


In [9]:
df.shape

(192, 1)

In [11]:
len_num = [len(num) for num in df["sequence"]]
df["len_num"] = len_num

In [12]:
RF_sequence = df[df["len_num"]>10]["sequence"]
transformer_sequence = df[df["len_num"]<11]["sequence"]

In [14]:
RF_sequence_representations = ESM_feature(RF_sequence)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<?, ?it/s]


In [15]:
transformer_sequence_representations = ESM_feature(transformer_sequence)

100%|████████████████████████████████████████████████████████████████████████████| 184/184 [00:00<00:00, 183444.72it/s]


# 分别产生特征和模型

In [16]:
import joblib
scaler_filename =r"model\Standard_scaler.save"
scaler = joblib.load(scaler_filename)

In [19]:
RF_pred = torch.stack(RF_sequence_representations)

In [20]:
transformer_X_pred = torch.stack(transformer_sequence_representations)

In [21]:
RF_X_pred = scaler.transform(RF_pred)

In [22]:
transformer_X_pred = scaler.transform(transformer_X_pred)

## 加载预测模型

In [24]:
class TransformerModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(TransformerModel, self).__init__()
        # 构建Transformer编码层，参数包括输入维度、注意力头数
        # 其中d_model要和模型输入维度相同
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=input_size,  # 输入维度
                                                        batch_first=True,
                                                        nhead=8)             # 注意力头数
        # 构建Transformer编码器，参数包括编码层和层数
        self.encoder = nn.TransformerEncoder(self.encoder_layer,             # 编码层
                                             num_layers=10)                   # 层数
        # 构建线性层，参数包括输入维度和输出维度（num_classes）
        self.fc = nn.Linear(input_size,                                      # 输入维度
                            num_classes)                                     # 输出维度


    def forward(self, x):
        #print("A:", x.shape)  # torch.Size([142, 13])
        x = x.unsqueeze(1)    # 增加一个维度，变成(batch_size, 1, input_size)的形状
        #print("B:", x.shape)  # torch.Size([142, 1, 13])
        x = self.encoder(x)   # 输入Transformer编码器进行编码
        #print("C:", x.shape)  # torch.Size([142, 1, 13])
        x = x.squeeze(1)      # 压缩第1维，变成(batch_size, input_size)的形状
        #print("D:", x.shape)  # torch.Size([142, 13])
        x = self.fc(x)        # 输入线性层进行分类预测
        #print("E:", x.shape)  # torch.Size([142, 3])
        return x

In [26]:
RF_model = joblib.load(r"model\RandomForest_classf.pkl")

D:\BaiduNetdiskDownload\Anaconda\envs\AHTPeptideFusion\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
D:\BaiduNetdiskDownload\Anaconda\envs\AHTPeptideFusion\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [27]:
model = torch.load(r'model\transformer_class.pt',map_location=torch.device('cuda:0'))#.to(device)

# 模型预测

In [28]:
RF_y = RF_model.predict(RF_pred)
RF_y_pro = RF_model.predict_proba(RF_pred)

In [29]:
RF_df = pd.DataFrame(np.array(RF_sequence),columns=["sequence"])
RF_df["Active_pro"] = RF_y_pro[:,0]
RF_df["Inactive_pro"] = RF_y_pro[:,1]

In [30]:
#extraT_df.to_excel("RF_ACE_true_prediction_result.xlsx",index=None)

In [31]:
#svc_df.to_excel("4_ACE_true_prediction_result.xlsx",index=None)

In [32]:
with torch.no_grad():
    model.eval()
    y_hat = model(torch.tensor(transformer_X_pred).float().to(device))   # 使用训练好的模型对测试集进行预测
    y_score = torch.softmax(y_hat, dim=1).data.cpu().numpy()
    prediction = torch.max(F.softmax(y_hat,dim=1), 1)[1]
    #pred_y = prediction.data.cpu().numpy().squeeze()

In [35]:
transformer_df = pd.DataFrame(np.array(transformer_sequence),columns=["sequence"])
transformer_df["Active_pro"] = y_score[:,0]
transformer_df["Inactive_pro"] = y_score[:,1]

In [36]:
#transformer_df.to_excel("prediction_result.xlsx",index=None)

In [37]:
df_all = pd.concat([RF_df,transformer_df], axis=0)

## 保存结果

In [38]:
df_all.to_excel("AHTPeptideFusion_prediction_result.xlsx",index=None)

In [39]:
sum([1 for x in df_all["Active_pro"] if x > 0.5])

176